In [1]:
# Manage all imports
import pandas as pd
import os
import datetime
import sys

# Dirty trick to be able to import common odis modules, if the notebook is not executed from 13_odis
current_dir = os.getcwd()
parent_dir = os.path.dirname(os.getcwd())
while not current_dir.endswith("13_odis"):
    print("changing to parent dir")
    os.chdir(parent_dir)
    current_dir = parent_dir
    parent_dir = os.path.dirname(current_dir)

print(os.getcwd())
sys.path.append(current_dir)

changing to parent dir
/Users/alex/dev/13_odis


In [2]:
# additional imports
from common.config import load_config
from common.data_source_model import DataSourceModel
from common.utils.file_handler import FileHandler
from common.utils.interfaces.data_handler import OperationType

In [ ]:
# define parameters for papermill

# model_name = "emploi.salaire_median"
# filepath = 'data/imports/emploi/emploi.salaire_median_1.zip'

model_name = "emploi.deplacement_domicile_travail"
filepath = 'data/imports/emploi/emploi.deplacement_domicile_travail_1.zip'

# model_name = "emploi.etablissements_employeurs_secteur_prive"
# filepath = 'data/imports/emploi/emploi.etablissements_employeurs_secteur_prive_1.zip'

In [4]:
# Initialize common variables
dataframes = {}
artifacts = []

config = load_config("datasources.yaml", response_model=DataSourceModel)
model = config.get_model( model_name = model_name )
start_time = datetime.datetime.now()

# Instantiate File Handler for file loads and dumps
handler = FileHandler()

In [ ]:
def get_subtable_name(dfname:str, model:DataSourceModel) -> str:

    subtable_name = ''

    long_name = f"{model.table_name}_{dfname.lower()}"

    print(len(long_name))

    if len(long_name) >= 63:
        print(f"Long table name exceeds the 63 characters limit: {long_name}")
        subtable_name = f"{model.domain_name}_{dfname.lower()}"
        print(f"Creating table with shorter name: {subtable_name}")
        print(f"Shorter table name length: {len(subtable_name)}")
    else:
        subtable_name = long_name

    return subtable_name

In [ ]:
from zipfile import ZipFile

# unzip and dump files into the data/imports folder
with open(filepath, 'rb') as f:
    zip_archive = ZipFile(f)

    zip_members = zip_archive.infolist()
    for member in zip_members:

        if not member.is_dir():

            member_filename = member.filename
            member_name = member_filename.split(".")[0]
            member_format = member_filename.split(".")[-1]
            print(member_format)
            
            f_member = zip_archive.open( member, 'r' ).read()
    
            artifact = handler.artifact_dump(
                f_member,
                get_subtable_name(member_name,model),
                model,
                format = member_format
            )

            print(artifact.model_dump(mode="yaml"))

            artifacts.append(artifact)

csv
2025-05-18 00:08:05,836 - DEBUG :: file_handler.py :: dump (130) :: dumping: data/imports/emploi/emploi.salaire_median_FILO2021_DEC_COM.csv


2025-05-18 00:08:05,915 - DEBUG :: file_handler.py :: file_dump (273) :: emploi.salaire_median -> results saved to : 'data/imports/emploi/emploi.salaire_median_FILO2021_DEC_COM.csv'
{'name': 'FILO2021_DEC_COM', 'storage_info': {'location': 'data/imports/emploi', 'format': 'csv', 'file_name': 'emploi.salaire_median_FILO2021_DEC_COM.csv', 'encoding': 'utf-8'}, 'load_to_bronze': True, 'success': True}
csv
2025-05-18 00:08:05,925 - DEBUG :: file_handler.py :: dump (130) :: dumping: data/imports/emploi/emploi.salaire_median_FILO2021_DEC_PAUVRES_COM.csv
2025-05-18 00:08:05,942 - DEBUG :: file_handler.py :: file_dump (273) :: emploi.salaire_median -> results saved to : 'data/imports/emploi/emploi.salaire_median_FILO2021_DEC_PAUVRES_COM.csv'
{'name': 'FILO2021_DEC_PAUVRES_COM', 'storage_info': {'location': 'data/imports/emploi', 'format': 'csv', 'file_name': 'emploi.salaire_median_FILO2021_DEC_PAUVRES_COM.csv', 'encoding': 'utf-8'}, 'load_to_bronze': True, 'success': True}
csv
2025-05-18 00:08

In [6]:
preprocess_metadata = handler.dump_metadata(
    model = model,
    operation = OperationType.PREPROCESS,
    start_time = start_time,
    complete = True,
    errors = 0,
    artifacts = artifacts,
    pages = []
)

2025-05-18 00:08:07,854 - DEBUG :: file_handler.py :: dump (130) :: dumping: data/imports/emploi/emploi.salaire_median_metadata_preprocess.json
2025-05-18 00:08:07,858 - DEBUG :: file_handler.py :: file_dump (273) :: emploi.salaire_median -> results saved to : 'data/imports/emploi/emploi.salaire_median_metadata_preprocess.json'
2025-05-18 00:08:07,859 - DEBUG :: file_handler.py :: dump_metadata (455) :: Metadata written in: 'data/imports/emploi/emploi.salaire_median_metadata_preprocess.json'


In [ ]:
from pathlib import Path

for artifact in artifacts:

    base_path = Path( artifact.storage_info.location )
    filepath = base_path / artifact.storage_info.file_name

    df = pd.read_csv(
        filepath,
        sep = ';',
        engine = 'python'
        )

    dataframes[ artifact.name ] = df


AttributeError: 'ArtifactLog' object has no attribute 'head'

In [ ]:
print("Loaded dataframes:")
last_key = ''
for key in dataframes.keys():
    last_key = key
    print(key)

dataframes[last_key].head()

Loaded dataframes:
FILO2021_DEC_COM
FILO2021_DEC_PAUVRES_COM
FILO2021_DISP_COM
FILO2021_DISP_PAUVRES_COM
FILO2021_TRDECILES_DEC_COM
FILO2021_TRDECILES_DISP_COM
meta_FILO2021_DEC_COM
meta_FILO2021_DEC_PAUVRES_COM
meta_FILO2021_DISP_COM
meta_FILO2021_DISP_PAUVRES_COM
meta_FILO2021_TRDECILES_DEC_COM
meta_FILO2021_TRDECILES_DISP_COM


,CODGEO,NBMEN21,NBPERS21,NBUC21,Q121,Q221,Q321,Q3_Q1,D121,D221,...,OPR6PTSA21,OPR6PCHO21,OPR6PBEN21,OPR6PPEN21,OPR6PPAT21,OPR6PPSOC21,OPR6PPFAM21,OPR6PPMINI21,OPR6PPLOGT21,OPR6PIMPOT21
0,01001,346,895,"590,8",s,25820,s,s,s,s,...,s,s,s,s,s,s,s,s,s,s
1,01002,115,266,"181,0",s,24480,s,s,s,s,...,s,s,s,s,s,s,s,s,s,s
2,01004,6855,15092,"10398,2",15800,21660,28430,12630,11890,14640,...,s,s,s,s,s,s,s,s,s,s
3,01005,800,2028,"1329,7",20010,24610,31180,11170,15560,18980,...,s,s,s,s,s,s,s,s,s,s
4,01006,51,107,"76,6",s,24210,s,s,s,s,...,s,s,s,s,s,s,s,s,s,s


In [18]:
from dotenv import dotenv_values
import sqlalchemy

# prepare db client
vals = dotenv_values()

conn_str = "postgresql://{}:{}@{}:{}/{}".format(
    vals["PG_DB_USER"],
    vals["PG_DB_PWD"],
    vals["PG_DB_HOST"],
    vals["PG_DB_PORT"],
    vals["PG_DB_NAME"]
)

dbengine = sqlalchemy.create_engine(conn_str)

In [ ]:
# insert all to bronze
# make the final table name lowercase to avoid issues in Postgre
for dfname, dataframe in dataframes.items():
    dataframe.to_sql(
        name = dfname,
        con = dbengine,
        schema = 'bronze',
        index = True,
        if_exists = 'replace'
    )
